# Mercedes-Benz Greener Manufacturing

### Following actions should be performed:
* If for any column(s), the variance is equal to zero, then you need to remove those variable(s).
* Check for null and unique values for test and train sets
* Apply label encoder.
* Perform dimensionality reduction.
* Predict your test_df values using xgboost

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
#warnings("ignore")

In [4]:
data = pd.read_csv(r"C:\Users\chhav\OneDrive\Desktop\Machine learning with python\Class Notes\Cource end Project\Mercedies Benz\train.csv")
data.head()

,ID,y,X0,X1,X2,X3,X4,X5,X6,X8,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,0,130.81,k,v,at,a,d,u,j,o,...,0,0,1,0,0,0,0,0,0,0
1,6,88.53,k,t,av,e,d,y,l,o,...,1,0,0,0,0,0,0,0,0,0
2,7,76.26,az,w,n,c,d,x,j,x,...,0,0,0,0,0,0,1,0,0,0
3,9,80.62,az,t,n,f,d,x,l,e,...,0,0,0,0,0,0,0,0,0,0
4,13,78.02,az,v,n,f,d,h,d,n,...,0,0,0,0,0,0,0,0,0,0


In [5]:
data.shape

(4209, 378)

In [6]:
## Removing column those have zero variance because we can't get any innformation from those variables
pd.options.display.float_format ='{:,.2f}'.format
variance = data.var(numeric_only = True)
variance

ID     5,941,936.12
y            160.77
X10            0.01
X11            0.00
X12            0.07
           ...     
X380           0.01
X382           0.01
X383           0.00
X384           0.00
X385           0.00
Length: 370, dtype: float64

In [7]:
variance = variance.reset_index()
variance

,index,0
0,ID,"5,941,936.12"
1,y,160.77
2,X10,0.01
3,X11,0.00
4,X12,0.07
...,...,...
365,X380,0.01
366,X382,0.01
367,X383,0.00
368,X384,0.00


In [8]:
variance.columns = {"Feature","Values"}
variance

,Feature,Values
0,ID,"5,941,936.12"
1,y,160.77
2,X10,0.01
3,X11,0.00
4,X12,0.07
...,...,...
365,X380,0.01
366,X382,0.01
367,X383,0.00
368,X384,0.00


In [9]:
### From the shapes, we infer the presence of 8 catagorical variable
var = variance.loc[variance["Values"]==0,"Feature"]
data1 = data.drop(var,axis=1)
print(data1.shape)
print(data1.head())
print(var)

(4209, 366)
   ID      y  X0 X1  X2 X3 X4 X5 X6 X8  ...  X375  X376  X377  X378  X379  \
0   0 130.81   k  v  at  a  d  u  j  o  ...     0     0     1     0     0   
1   6  88.53   k  t  av  e  d  y  l  o  ...     1     0     0     0     0   
2   7  76.26  az  w   n  c  d  x  j  x  ...     0     0     0     0     0   
3   9  80.62  az  t   n  f  d  x  l  e  ...     0     0     0     0     0   
4  13  78.02  az  v   n  f  d  h  d  n  ...     0     0     0     0     0   

   X380  X382  X383  X384  X385  
0     0     0     0     0     0  
1     0     0     0     0     0  
2     0     1     0     0     0  
3     0     0     0     0     0  
4     0     0     0     0     0  

[5 rows x 366 columns]
3       X11
83      X93
97     X107
219    X233
221    X235
254    X268
275    X289
276    X290
279    X293
283    X297
315    X330
332    X347
Name: Feature, dtype: object


In [10]:
data1.drop("ID",axis=1,inplace=True)
data1.shape

(4209, 365)

In [11]:
## Check for null values and unique values for the train and test sets:
data1[data1.isnull().any(axis=1)]

,y,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385


In [12]:
test = pd.read_csv(r"C:\Users\chhav\OneDrive\Desktop\Machine learning with python\Class Notes\Cource end Project\Mercedies Benz\test.csv")
test.head()

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385
0,1,az,v,n,f,d,t,a,w,0,...,0,0,0,1,0,0,0,0,0,0
1,2,t,b,ai,a,d,b,g,y,0,...,0,0,1,0,0,0,0,0,0,0
2,3,az,v,as,f,d,a,j,j,0,...,0,0,0,1,0,0,0,0,0,0
3,4,az,l,n,f,d,z,l,n,0,...,0,0,0,1,0,0,0,0,0,0
4,5,w,s,as,c,d,y,i,m,0,...,1,0,0,0,0,0,0,0,0,0


In [13]:
test[test.isnull().any(axis=1)]

,ID,X0,X1,X2,X3,X4,X5,X6,X8,X10,...,X375,X376,X377,X378,X379,X380,X382,X383,X384,X385


In [14]:
catagorical = data1.select_dtypes(exclude='number')
numerical = data1.select_dtypes(include='number')

In [15]:
catagorical.columns

Index(['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8'], dtype='object')

In [16]:
# catagorical['X0'].unique() instead of doing this we go for a loop

In [17]:
for j in catagorical:
    print(catagorical[j].unique())
    print(j,"---"*40)

['k' 'az' 't' 'al' 'o' 'w' 'j' 'h' 's' 'n' 'ay' 'f' 'x' 'y' 'aj' 'ak' 'am'
 'z' 'q' 'at' 'ap' 'v' 'af' 'a' 'e' 'ai' 'd' 'aq' 'c' 'aa' 'ba' 'as' 'i'
 'r' 'b' 'ax' 'bc' 'u' 'ad' 'au' 'm' 'l' 'aw' 'ao' 'ac' 'g' 'ab']
X0 ------------------------------------------------------------------------------------------------------------------------
['v' 't' 'w' 'b' 'r' 'l' 's' 'aa' 'c' 'a' 'e' 'h' 'z' 'j' 'o' 'u' 'p' 'n'
 'i' 'y' 'd' 'f' 'm' 'k' 'g' 'q' 'ab']
X1 ------------------------------------------------------------------------------------------------------------------------
['at' 'av' 'n' 'e' 'as' 'aq' 'r' 'ai' 'ak' 'm' 'a' 'k' 'ae' 's' 'f' 'd'
 'ag' 'ay' 'ac' 'ap' 'g' 'i' 'aw' 'y' 'b' 'ao' 'al' 'h' 'x' 'au' 't' 'an'
 'z' 'ah' 'p' 'am' 'j' 'q' 'af' 'l' 'aa' 'c' 'o' 'ar']
X2 ------------------------------------------------------------------------------------------------------------------------
['a' 'e' 'c' 'f' 'd' 'b' 'g']
X3 -------------------------------------------------------------------

In [18]:
character = test.select_dtypes(exclude='number')
numerical = test.select_dtypes(include='number')
character.columns

Index(['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8'], dtype='object')

In [19]:
for j in character:
    print(character[j].unique())
    print(j,"---"*40)

['az' 't' 'w' 'y' 'x' 'f' 'ap' 'o' 'ay' 'al' 'h' 'z' 'aj' 'd' 'v' 'ak'
 'ba' 'n' 'j' 's' 'af' 'ax' 'at' 'aq' 'av' 'm' 'k' 'a' 'e' 'ai' 'i' 'ag'
 'b' 'am' 'aw' 'as' 'r' 'ao' 'u' 'l' 'c' 'ad' 'au' 'bc' 'g' 'an' 'ae' 'p'
 'bb']
X0 ------------------------------------------------------------------------------------------------------------------------
['v' 'b' 'l' 's' 'aa' 'r' 'a' 'i' 'p' 'c' 'o' 'm' 'z' 'e' 'h' 'w' 'g' 'k'
 'y' 't' 'u' 'd' 'j' 'q' 'n' 'f' 'ab']
X1 ------------------------------------------------------------------------------------------------------------------------
['n' 'ai' 'as' 'ae' 's' 'b' 'e' 'ak' 'm' 'a' 'aq' 'ag' 'r' 'k' 'aj' 'ay'
 'ao' 'an' 'ac' 'af' 'ax' 'h' 'i' 'f' 'ap' 'p' 'au' 't' 'z' 'y' 'aw' 'd'
 'at' 'g' 'am' 'j' 'x' 'ab' 'w' 'q' 'ah' 'ad' 'al' 'av' 'u']
X2 ------------------------------------------------------------------------------------------------------------------------
['f' 'a' 'c' 'e' 'd' 'g' 'b']
X3 --------------------------------------------------

#### Apply label encoder

In [21]:
from sklearn.preprocessing import LabelEncoder

In [22]:
for col in catagorical.columns:
    label_encoder = LabelEncoder()
    data1[col] = label_encoder.fit_transform(data1[col])
    

In [23]:
data1.dtypes

y       float64
X0        int32
X1        int32
X2        int32
X3        int32
         ...   
X380      int64
X382      int64
X383      int64
X384      int64
X385      int64
Length: 365, dtype: object

In [24]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4209 entries, 0 to 4208
Columns: 365 entries, y to X385
dtypes: float64(1), int32(8), int64(356)
memory usage: 11.6 MB


In [25]:
for char in character.columns:
   label_encoder = LabelEncoder()
   data1[char] = label_encoder.fit_transform(data1[char])

In [26]:
train_data_feature = data1.drop(columns = {'y'})
train_data_target = data1.y
print(train_data_feature.shape)

(4209, 364)


In [27]:
### Apply Dimention reduction
from sklearn.decomposition import PCA
pca = PCA(n_components =.95)
pca.fit(train_data_feature,train_data_target)

PCA(n_components=0.95)

In [28]:
train_data_feature_transform = pca.fit_transform(train_data_feature)
train_data_feature_transform.shape

(4209, 6)

In [29]:
### XGBoost 
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error
from math import sqrt

In [30]:
train_x,test_x,train_y,test_y = train_test_split(train_data_feature_transform,train_data_target,test_size =0.30,random_state = 123)
print(train_x.shape)
print(test_x.shape)
print(train_y.shape)
print(test_y.shape)

(2946, 6)
(1263, 6)
(2946,)
(1263,)


In [31]:
xgb_reg = xgb.XGBRegressor(objective = 'reg:squarederror',n_estimators = 100, seed=123)
xgb_reg.fit(train_x,train_y)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=100,
             n_jobs=None, num_parallel_tree=None, ...)

In [32]:
## predict
y_pred = xgb_reg.predict(test_x)

In [33]:
d = pd.DataFrame()
d["Actuals"] = test_y
d["Predictions"] = y_pred
d['AB_Error'] = abs((d["Actuals"]-d["Predictions"])/d["Actuals"])
d

,Actuals,Predictions,AB_Error
1259,105.97,104.87,0.01
1793,89.28,92.78,0.04
2720,98.84,95.51,0.03
2024,80.20,78.85,0.02
3578,98.10,98.46,0.00
...,...,...,...
2362,87.13,95.04,0.09
3382,91.17,94.60,0.04
3404,88.28,99.66,0.13
1464,96.61,103.60,0.07


In [34]:
MAPE = (d['AB_Error'].mean())*100
MAPE

7.220067681439686

### Prediction on test Data

In [36]:
test.columns

Index(['ID', 'X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8', 'X10',
       ...
       'X375', 'X376', 'X377', 'X378', 'X379', 'X380', 'X382', 'X383', 'X384',
       'X385'],
      dtype='object', length=377)

In [37]:
character.columns

Index(['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8'], dtype='object')

In [38]:
test = test.drop(columns ={ 'X11', 'X93', 'X107', 'X233', 'X235', 'X268', 'X289', 'X290', 'X293', 'X297', 'X330', 'X347'})

In [39]:
test.shape

(4209, 365)

In [40]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [41]:
for col in character.columns:
    label_encoder = LabelEncoder()
    test[col] = label_encoder.fit_transform(test[col])
    

In [42]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4209 entries, 0 to 4208
Columns: 365 entries, ID to X385
dtypes: int32(8), int64(357)
memory usage: 11.6 MB


In [43]:
test.shape

(4209, 365)

In [44]:
test_data_feature = test.drop("ID",axis = 1)
test_data_feature.shape

(4209, 364)

In [45]:
pca.fit(test_data_feature)

PCA(n_components=0.95)

In [46]:
test_data_feature = pca.fit_transform(test_data_feature)
print(test_data_feature.shape)

(4209, 6)


In [47]:
test.shape

(4209, 365)

In [48]:
test_pred = xgb_reg.predict(test_data_feature)
Result_2d = test_pred.reshape(-1,1)
Result_2d

array([[ 81.16529 ],
       [ 93.338524],
       [ 86.41394 ],
       ...,
       [ 99.643036],
       [107.512505],
       [ 91.208084]], dtype=float32)

In [49]:
### Saving the array in csv file
Result_2d.tofile("result1.csv",sep =",")

In [50]:
## Export result in dataframe format:

result_df = pd.DataFrame({"Predictions":test_pred})
result_df

,Predictions
0,81.17
1,93.34
2,86.41
3,82.18
4,104.24
...,...
4204,118.01
4205,93.86
4206,99.64
4207,107.51


In [51]:
result_df.to_csv("tabular_output.csv")